In [53]:
import pandas as pd
import numpy as np

In [116]:
# read all the excel files and combine in one file. give the names of domain based on filename
import os
l=os.listdir()
df_l=[]
for i in l:
    if i.startswith('7'):
        df=pd.read_excel(i,sheet_name=None)
        df=df[list(df.keys())[1]]
        name=i.split('.')[0].split('_')[0][1::]
        df['domain']=name
        df_l.append(df)

df=pd.concat(df_l)

In [60]:
ng=pd.read_csv('ngrams_list.csv')  # read the ngrams list
ng['domain']=ng['domain'].astype(str).str.lower() # make the ngram and domain columns lowercase
ng['ngram']=ng['ngram'].astype(str).str.lower()

In [108]:
## Following function will search for the ngram in the utterance and then multiply the weight in top_p1, top_p2 and top_p3 columns if found.

def fun(row):
    d={1:1.25,2:1.5,3:1.75,0:1,4:1,5:1} # multiply with 1.25 if length of ngram is 1, with 1.5 if length is 2 and 1.75 if its 3.
    l=[]
    n=[]
    try:
        ng1=ng[ng['domain']==row['domain']].drop_duplicates(subset=['ngram'])  # select the corresponding domain in the ngram
        for i in ng1['ngram']:
            if str(i) in str(row['utterance']):
                l.append(len(str(i).split()))
                n.append(str(i))

        l1=max(l)   # take the maximum of the lengths of the ngrams so that if multiple ngrams are found prefer with largest length.
        n1=n[np.argmax(l)] # take the matched ngram
    except:
        l1=0
        n1='None'  # put none if ngram is not found in the utterance
    row['top_p1_updated']=row['top_p1']*d[l1]
    row['top_p2_updated']=row['top_p2']*d[l1]
    row['top_p3_updated']=row['top_p3']*d[l1]
    row['ngram_found']=n1
    return row

In [109]:
import swifter

df1=df.swifter.apply(lambda x: fun(x),axis=1)

In [112]:
df1.to_excel('output.xlsx',index=None)